In [1]:
print("I am fine")

I am fine


In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
from transformers import LlamaForCausalLM, LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("/120040051/hf_llama2")
model = LlamaForCausalLM.from_pretrained("/120040051/hf_llama2").to(device)

/root/anaconda3/envs/llama2-py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]/root/anaconda3/envs/llama2-py311/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 3/3 [00:21<00:00,  7.29s/it]


In [4]:
def calculate_perplexity(sentences, tokenizer, model, device):
    model.eval()  # Ensure the model is in evaluation mode
    perplexities = []

    for sentence in sentences:
        # Tokenize the input sentence
        encodings = tokenizer(sentence, return_tensors="pt")

        # Move encodings to the same device as the model
        input_ids = encodings.input_ids.to(device)
        target_ids = input_ids.clone()
        attention_mask = encodings.attention_mask.to(device)

        # Calculate Negative Log-Likelihood
        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids, attention_mask=attention_mask)
            neg_log_likelihood = outputs.loss * input_ids.size(1)  # Multiply by sequence length

        # Calculate Perplexity and append to list
        ppl = torch.exp(neg_log_likelihood / input_ids.size(1))  # Divide by sequence length
        perplexities.append(ppl.item())

    return perplexities

In [5]:
sentences = ["A man is eating pizza in the restaurant.", "A man is drinking beer in the restaurant.", "A man is play golf in the restaurant."]
perplexity_values = calculate_perplexity(sentences, tokenizer, model, device)
print(perplexity_values)

[21.415712356567383, 20.11915397644043, 151.33119201660156]
